In [3]:
from unsloth import FastLanguageModel
from datasets import load_from_disk
from trl import SFTTrainer
from transformers import TrainingArguments
import torch
import wandb

In [12]:
class Lora_FineTuner:
    """
    QLORA Fine-tuning using Unsloth, Huggingface Transformers.
    """
    
    def __init__(self, dataset, model_name: str, max_seq_length: int, 
                 load_in_4bit: bool = True, wandb_track: bool = True) -> None:
        self.model_name = model_name
        self.dataset = load_from_disk(dataset)
        self.max_seq_length = max_seq_length
        self.load_in_4bit = load_in_4bit
        if wandb_track: self._setup_wandb()
        self._setup_model_and_tokenizer()
    
    def _setup_wandb(self):
        pass
        
    
    def _setup_model_and_tokenizer(self):
        model, self.tokenizer = FastLanguageModel.from_pretrained(
        model_name = self.model_name, 
        max_seq_length = self.max_seq_length,
        dtype = None,
        load_in_4bit = self.load_in_4bit,
        )

        self.model = FastLanguageModel.get_peft_model(
            model,
            r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                            "gate_proj", "up_proj", "down_proj",],
            lora_alpha = 16,
            lora_dropout = 0, # Supports any, but = 0 is optimized
            bias = "none",    # Supports any, but = "none" is optimized
            use_gradient_checkpointing = True,
            random_state = 3407,
            use_rslora = False,
            loftq_config = None,
        )
    
    def _setup_trainer(self):
        self.trainer = SFTTrainer(
            model = self.model,
            tokenizer = self.tokenizer,
            train_dataset = self.dataset['train'],
            eval_dataset = self.dataset['eval'],
            dataset_text_field = "text",
            max_seq_length = self.max_seq_length,
            dataset_num_proc = 2,
            packing = False, # Can make training 5x faster for short sequences.
            args = TrainingArguments(
                per_device_train_batch_size = 2,
                gradient_accumulation_steps = 4,
                warmup_steps = 5,
                learning_rate = 2e-4,
                num_train_epochs=2,
                fp16 = not torch.cuda.is_bf16_supported(),
                bf16 = torch.cuda.is_bf16_supported(),
                logging_steps = 500,
                optim = "adamw_8bit",
                weight_decay = 0.01,
                eval_steps=1000,
                do_eval = True,
                lr_scheduler_type = "linear",
                seed = 3407,
                report_to = "wandb",
                output_dir = "outputs",
            ),
        )
    
    def train(self):
        self._setup_trainer()
        trainer_stats = self.trainer.train()


In [13]:
wandb.login()

True

In [14]:
llm_trainer = Lora_FineTuner(dataset="arcelik_segmentation_data_sample",
                             model_name="unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
                             max_seq_length=130
                             )

==((====))==  Unsloth: Fast Mistral patching release 2024.2
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.731 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.24. FA = False.
 "-____-"     Apache 2 free license: http://github.com/unslothai/unsloth


/workspace/llm-fine-tuning/venv/lib/python3.10/site-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Unsloth 2024.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [15]:
llm_trainer.train()

Map (num_proc=2): 100%|██████████| 15/15 [00:00<00:00, 30.96 examples/s]
/workspace/llm-fine-tuning/venv/lib/python3.10/site-packages/accelerate/accelerator.py:437: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 84 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch 

Step,Training Loss
